### Eksempler
<font size=2>Vi tar utgangspunkt i datasettet med landenes innbyggertall.
<ol>
<li>Lese inn og vise dataframe
<li>Sortere dataframe etter variabel
<li>Telle opp antall duplikater i form av identiske rader i dataframen og vise i liste
<li>Telle opp anntall duplikater i en variabel
<li>ny dataframe hvor duplikater på land er fjernet (dvs det skal gjenstå en rad pr land)- Fjerne dubletter, beholde en rad basert på et gitt kriterie
<li>Lage ny dataframe hvor duplikater på land er fjernet. Den som gjenstår skal være den rad med høyest innbyggerantall
<li>Ta ut de rader som har to eller flere forekomster på samme land i egen dataframe
<li>Telle opp antall og skrive ut/printe hvor mange rader som er identiske (duplikat på alle dvs variablene er identiske)
<li>Lage ny dataframe hvor rader som er identiske fjernet (dvs det skal gjenstå en rad) (edited)
</ol>
</font>

#### Importerer biblioteker

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as F

### 1. Lese inn og vise dataframe 
<font size=2>Lese inn dataframe: <code><i>navn_dataframe</i>.spark.read.path("/felles/sti")</code>
Vise dataframe: <code><i>navn_dataframe</i>.show()</code>
</font>

#### Leser inn og viser dataframen "df_innbyggertall" (kode)

In [ ]:
#Her leses dataframen inn fra dataplattformen som vi ønsker å undersøke nærmere for dubletter

df_innbyggerantall = spark.read.path("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")

# Her vises dataframen vi har lest inn 
df_innbyggerantall.show(30, False)

### 2. Sortere dataframe etter kolonne

<code><i>dataframe</i>.orderBy('kolonnenavn')</code>

#### Lager nytt dataframe 'df_innbyggerantall_sort' som er sortert etter variabelen 'Land' (kode)

In [ ]:
#Sorterer dataframen etter variabelen "land" alfabetisk
df_innbyggerantall_sort = df_innbyggerantall.orderBy('Land')
#lister ut den sorterte dataframen
df_innbyggerantall_sort.show()

### 3.Telle opp antall duplikater  dataframen og vise i liste
<font size=2><code>'dataframe'.groupBy('dataframe'.columns)\
    .count()\
    .where(F.col('count') > 1)\
    .select(F.sum('count'))
    .show()</code>
</font>

#### Telle opp antall dublikater og vise i liste (kode)

In [ ]:
#Sjekke og telle om det er duplikater i form av identiske rader i dataframe. 

df_innbyggerantall_sort.groupBy(df_innbyggerantall_sort.columns)\
    .count()\
    .where(F.col('count') > 1)\
    .select(F.sum('count'))\
    .show()

### 4. Telle opp antall dublikater i en kolonne

#### Antall duplikater i variabelen 'Land' i dataframen 'df_innbyggerantall_sort' (kode)

In [ ]:
# Telle opp antall duplikater i variabelen "Land" i dataframen. 
df_innbyggerantall_sort.groupBy(df_innbyggerantall_sort.Land)\
    .count()\
    .where(F.col('count') > 1)\
    .select(F.sum('count'))\
    .show()

#### Lager ny dataframe 'df_land_dup' med dublettene i variabelen 'land' (kode)

In [ ]:
#lager ny dataframe med bestående av land og tilhørende duplikater. 

df_land_dup=df_innbyggerantall_sort.groupBy(df_innbyggerantall_sort.Land)\
    .count()\
    .filter(F.col('count') > 1)
df_land_dup.show()

In [ ]:
#Tar ut oversikt over de radene i kolonnen "Land" som er duplikater
df_innbyggerantall_sort.exceptAll(df_innbyggerantall_sort.dropDuplicates(['Land'])).show()

In [ ]:
#Tar ut oversikt over de radene med likt "land" som forekommer flere enn en gang i dataframen, og sånn sett utgjør den samlede mengden av duplikater
df_land_dub_alle  = df_innbyggerantall_sort.join(df_innbyggerantall_sort.groupBy('Land')\
          .count().where('count = 1').drop('count'),
        on=['Land'],
        how='left_anti')
df_land_dub_alle.show()        

### Alternativ

In [ ]:
df_antall_dublikater= df_innbyggerantall.groupBy('Land').count()

df_antall_dublikater=df_antall_dublikater.filter((F.col('count')>1))

print("Antall dublikater:")
df_antall_dublikater.show()

dublikater = df_innbyggerantall.join(df_antall_dublikater,'Land','inner')

print("Dublikatene:")
dublikater.show()

In [ ]:
#Sorterer duplikat-dataframe etter land og innbyggerantall
df_land_dub_alle.orderBy(['Land','Innbyggerantall']).show()

### Å fjerne dublikater


In [ ]:
#Fjerner/dropper rader som er helt like
df_innbyggerantall_udub=df_innbyggerantall.dropDuplicates()
df_innbyggerantall_udub.show()
df_innbyggerantall_udub.count()
#


In [ ]:
#Fjerner/dropper rader med duplikat på land
df_innbyggerantall_udubLand=df_innbyggerantall.dropDuplicates(['Land'])
df_innbyggerantall_udubLand.show()
df_innbyggerantall_udubLand.count()

In [ ]:
#Velger den første duplikatene
#Overfører df_land_dub_alle til pandas dataframe (pdf)
pdf = df_land_dub_alle.toPandas()
df_pdf_udub = pdf.drop_duplicates(subset=['Land'], keep='first', inplace=False)

#Overfører "tilbake" til spark dataframe
schema_land_innbyggerantall = StructType([\
    StructField('Land',StringType(), False),\
    StructField('Landkode',StringType(), False),\
    StructField('Innbyggerantall', IntegerType(), False),\
    StructField('År', IntegerType(), False),\
    StructField('Kilde', StringType(), False)])

df_spark_udub = spark.createDataFrame(df_pdf_udub,  schema_land_innbyggerantall)
df_spark_udub.show()